In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### Imports

In [ ]:
import numpy as np 
import pandas as pd
import transformers
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from datasets import Dataset
from transformers import TrainingArguments, Trainer
from sklearn.metrics import mean_squared_error
import torch
from sklearn.model_selection import KFold, GroupKFold

In [ ]:
import random
import os
import warnings
import logging
import shutil
from tqdm import tqdm
from datasets import disable_progress_bar
warnings.simplefilter("ignore")
logging.disable(logging.ERROR)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
disable_progress_bar()
tqdm.pandas()

In [ ]:
class CFG:
    model_name="debertav3base"
    learning_rate=1.5e-5
    weight_decay=0.02
    hidden_dropout_prob=0.007
    attention_probs_dropout_prob=0.007
    num_train_epochs=5
    n_splits=4
    batch_size=8
    random_seed=42
    save_steps=500
    max_length=512

In [ ]:
# SEED 42
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(CFG.random_seed)

## Load Data

In [ ]:
DATA_DIR = "/kaggle/input/commonlit-evaluate-student-summaries/"

prompts_train = pd.read_csv(DATA_DIR + "prompts_train.csv")
prompts_test = pd.read_csv(DATA_DIR + "prompts_test.csv")
summaries_train = pd.read_csv(DATA_DIR + "summaries_train.csv")
summaries_test = pd.read_csv(DATA_DIR + "summaries_test.csv")
sample_submission = pd.read_csv(DATA_DIR + "sample_submission.csv")

In [ ]:
train = pd.merge(prompts_train,summaries_train,on='prompt_id')
test=pd.merge(prompts_test,summaries_test,on='prompt_id')

In [ ]:
train.head(2)

In [ ]:
test.head(2)

#### Split train data using GroupKFolds on prompt_id
Since test dataset will have new prompts, hence task becomes to train model to perform well on new/unseen prompts - https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/discussion/425409#2357563

In [ ]:
gkf = GroupKFold(n_splits=CFG.n_splits) # Since 4 prompts in training set

for i, (_, val_index) in enumerate(gkf.split(train, groups=train["prompt_id"])):
    train.loc[val_index, "fold"] = i

In [ ]:
train.groupby("fold").count()

#### Prepare huggingface dataset
Ref - https://huggingface.co/docs/datasets/v2.14.5/en/tabular_load#pandas-dataframes

In [ ]:
target_cols = ['content','wording']
text_cols = ['text']

In [ ]:
train_dataset = Dataset.from_pandas(train[text_cols + target_cols + ['fold']])
test_dataset = Dataset.from_pandas(test[text_cols])

In [ ]:
train_dataset, test_dataset

## Define Model and Metrics

#### Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(f'/kaggle/input/{CFG.model_name}')

In [ ]:
tokenizer.encode_plus(train_dataset[0]['text'])

In [ ]:
train_dataset[0]

In [ ]:
## Figure out generating labels columns for batch of thousand
def generate_tokens(examples: pd.DataFrame,mode='train'):
    encodings = tokenizer(examples['text'],truncation=True,max_length=CFG.max_length,return_tensors='np')
    if mode == 'test':
        return encodings
    labels = np.column_stack((examples['content'],examples['wording']))
    return {**encodings, "labels": labels}

tokenized_train_dataset = train_dataset.map(generate_tokens,batched=True)

In [ ]:
tokenized_train_dataset

In [ ]:
tokenized_test_dataset = test_dataset.map(lambda x: generate_tokens(x,'test'),batched=True)

In [ ]:
tokenized_test_dataset

#### Config

In [ ]:
config = AutoConfig.from_pretrained(f'/kaggle/input/{CFG.model_name}')

In [ ]:
config

In [ ]:
config.update({
    "num_labels": 2,
    "problem_type": 'regression',
    "hidden_dropout_prob": CFG.hidden_dropout_prob,
    "attention_probs_dropout_prob": CFG.attention_probs_dropout_prob
})

In [ ]:
config

#### Model

In [ ]:
# model = AutoModel.from_pretrained(f'/kaggle/input/{CFG.model_name}',config=config)

In [ ]:
# model

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(f'/kaggle/input/{CFG.model_name}',config=config)

In [ ]:
model

> Can see that a pooling layer followed by Linear Layer with 2 outputs was added to base model

#### Metrics - MCRMSE

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

def compute_mcrmse(eval_pred):
    """
    Calculates mean columnwise root mean squared error
    https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/overview/evaluation
    """
    preds, labels = eval_pred

    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }

def compt_score(content_true, content_pred, wording_true, wording_pred):
    content_score = mean_squared_error(content_true, content_pred)**(1/2)
    wording_score = mean_squared_error(wording_true, wording_pred)**(1/2)
    
    return (content_score + wording_score)/2

## Train using GroupKFold for CV

<!-- ### Build Trainer -->

In [ ]:
train_collator = DataCollatorWithPadding(tokenizer)

In [ ]:
# model_fold_dir = './'
# training_args = TrainingArguments(
#     output_dir = model_fold_dir,
#     report_to='none',
#     load_best_model_at_end=True, # select best model
#     learning_rate=CFG.learning_rate,
#     per_device_train_batch_size=CFG.batch_size,
#     per_device_eval_batch_size=CFG.batch_size,
#     num_train_epochs=CFG.num_train_epochs,
#     weight_decay=CFG.weight_decay,
#     greater_is_better=False,
#     metric_for_best_model="mcrmse",
#     save_strategy='no', # "steps",
#     evaluation_strategy='no' #"steps",
# )
## report_to='none' to avoid wandb login - https://discuss.huggingface.co/t/how-to-turn-wandb-off-in-trainer/6237/2
## both save strategy and eval strategy have to match

#### GPU

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
cuda.empty_cache()
print(device)

In [ ]:
# model_gpu = model.to(device)

In [ ]:
# model_gpu

In [ ]:
# trainer = Trainer(
#     model = model_gpu,
#     train_dataset = tokenized_train_dataset,
#     args = training_args,
#     data_collator = train_collator,
#     tokenizer = tokenizer,
#     compute_metrics = compute_mcrmse    
# )

In [ ]:
import gc
gc.collect()

#### Training loop

In [ ]:
# delete old model files
if os.path.exists(CFG.model_name):
    shutil.rmtree(CFG.model_name)
os.mkdir(CFG.model_name)

for fold in range(CFG.n_splits):
    print(f"Fold: {fold}")
    fold_train_data = train[train['fold']!=fold]
    fold_val_data = train[train['fold']==fold]
    fold_train_dataset = Dataset.from_pandas(fold_train_data[text_cols + target_cols])
    fold_val_dataset = Dataset.from_pandas(fold_val_data[text_cols + target_cols])
    fold_train_tokenized = fold_train_dataset.map(generate_tokens,batched=True)
    fold_val_tokenized = fold_val_dataset.map(generate_tokens,batched=True)
    print(f"Number of training examples: {fold_train_tokenized.num_rows}")
    print(f"Number of validation examples: {fold_val_tokenized.num_rows}")
    
    model = AutoModelForSequenceClassification.from_pretrained(f'/kaggle/input/{CFG.model_name}',config=config)
    model_gpu = model.to(device)
    
    model_dir = f"{CFG.model_name}/fold_{fold}"
    model_fold_dir = os.path.join(model_dir, str(fold))
    
    training_args = TrainingArguments(
        output_dir = model_fold_dir,
        report_to='none',
        load_best_model_at_end=True, # select best model
        learning_rate=CFG.learning_rate,
        per_device_train_batch_size=CFG.batch_size,
        per_device_eval_batch_size=CFG.batch_size,
        num_train_epochs=CFG.num_train_epochs,
        weight_decay=CFG.weight_decay,
        greater_is_better=False,
        metric_for_best_model="mcrmse",
        save_strategy='steps',
        evaluation_strategy='steps',
        save_total_limit=1,
        fp16=True,
        save_steps = CFG.save_steps,
        eval_steps = CFG.save_steps
    )
    
    trainer = Trainer(
        model = model_gpu,
        train_dataset = fold_train_tokenized,
        eval_dataset = fold_val_tokenized,
        args = training_args,
        data_collator = train_collator,
        tokenizer = tokenizer,
        compute_metrics = compute_mcrmse    
    )
    
    trainer.train()
    
    model_gpu.save_pretrained(model_dir)
    tokenizer.save_pretrained(model_dir)
    model_gpu.cpu()
    del model
    gc.collect()
    torch.cuda.empty_cache()

## Predict

In [ ]:
# preds = trainer.predict(tokenized_test_dataset)

In [ ]:
# preds

In [ ]:
# test_args = TrainingArguments(
#             output_dir=  model_fold_dir,
#             do_train = False,
#             do_predict = True,
#             per_device_eval_batch_size = 4,   
#             dataloader_drop_last = False
# )

In [ ]:
# infer = Trainer(
#     model = model_gpu,
#     args = test_args,
#     tokenizer = tokenizer,
#     data_collator = train_collator
# )

In [ ]:
# preds2 = infer.predict(tokenized_test_dataset)

In [ ]:
# preds2[0]

In [ ]:
for fold in range(CFG.n_splits):
    model_dir = f"{CFG.model_name}/fold_{fold}"
    model_fold_dir = os.path.join(model_dir, str(fold))

    model = AutoModelForSequenceClassification.from_pretrained(model_dir)    
    model.eval() # Set default model mode to evaluation - https://huggingface.co/docs/transformers/main_classes/model
    model_gpu = model.to(device)
    
    test_args = TrainingArguments(
        output_dir=  model_fold_dir,
        do_train = False,
        do_predict = True,
        per_device_eval_batch_size = 4,   
        dataloader_drop_last = False,
        fp16=True
    )
    
    infer = Trainer(
        model = model_gpu,
        args = test_args,
        tokenizer = tokenizer,
        data_collator = train_collator
    )
    
    preds = infer.predict(tokenized_test_dataset)[0]
    test[f"{target_cols[0]}_{fold}"] = preds[:,0]
    test[f"{target_cols[1]}_{fold}"] = preds[:,1]
    
    model_gpu.cpu()
    del model_gpu
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
test

In [ ]:
## Take mean of predictions across all folds
test[target_cols[0]] = test[[f"{target_cols[0]}_{fold}" for fold in range(CFG.n_splits)]].mean(axis=1)
test[target_cols[1]] = test[[f"{target_cols[1]}_{fold}" for fold in range(CFG.n_splits)]].mean(axis=1)

## Submission

In [ ]:
df_submission = pd.DataFrame()
df_submission['student_id'] = test['student_id']
df_submission['content'] = 0
df_submission['wording'] = 0
df_submission[target_cols[0]] = test[target_cols[0]]
df_submission[target_cols[1]] = test[target_cols[1]]
df_submission.to_csv('submission.csv', index=False)

In [ ]:
df_submission